In [77]:
# %cd /home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux 
# %cd test
# !rm -rf *.jpg
# !rmdir unknown


# %cd /home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux 
# %cd train
# !rm -rf *.jpg
# !rm -rf sample

# %cd /home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux 
# !rm -rf valid
# !rm -rf results
# !rm -rf sample/train/
# !rm -rf sample/valid/

# %cd /home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux 
# !rmdir test
# !rmdir train


# !unzip -q test.zip
# !unzip -q train.zip

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux
/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/test
/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux
/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/train
/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux
/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux


In [78]:
%cd /home/ubuntu/courses/deeplearning1/nbs/lesson1
%pwd

/home/ubuntu/courses/deeplearning1/nbs/lesson1


u'/home/ubuntu/courses/deeplearning1/nbs/lesson1'

In [79]:
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir + '/data/redux'

In [80]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from utils import *
from vgg16 import Vgg16

%matplotlib inline

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories
3. Finetune and Train model
4. Generate predictions
5. Validation predictions
6. Submit predictions to Kaggle

## Create validation set and sample

In [81]:
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux


In [82]:
!tree -d

.
├── results
├── sample
│   ├── results
│   ├── test
│   ├── train
│   └── valid
├── test
│   └── unknown
├── train
└── valid

10 directories


In [83]:
%cd $DATA_HOME_DIR/train

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/train


In [84]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000) : os.rename(shuf[i], DATA_HOME_DIR + '/valid/' + shuf[i])

In [85]:
!echo ----Valid----
%ls ../valid | head -5 && ls ../valid  | tail -5 && ls ../valid  | wc -l
%ls | wc -l
# 2000 random files moved from /train to /valid
# /valid has 2000 files
# Now /train has 25000 - 2000 = 23000

----Valid----
cat.10004.jpg
cat.10035.jpg
cat.10036.jpg
cat.1003.jpg
cat.10045.jpg
ls: write error
dog.9910.jpg
dog.9920.jpg
dog.997.jpg
dog.9980.jpg
dog.9999.jpg
2000
23000


In [86]:
from shutil import copyfile

In [87]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200) : copyfile(shuf[i], DATA_HOME_DIR + '/sample/train/' + shuf[i])

In [88]:
# Copied 200 files from /train to /sample/train
%ls ../sample/train/ | head -5 && ls ../sample/train/ | tail -5 && ls ../sample/train/| wc -l
%ls | wc -l
# /sample/train has 200 files
# /train has 23000 files

cat.10003.jpg
cat.10037.jpg
cat.10151.jpg
cat.10262.jpg
cat.10580.jpg
dog.9645.jpg
dog.9697.jpg
dog.9700.jpg
dog.9884.jpg
dog.996.jpg
200
23000


In [89]:
%cd $DATA_HOME_DIR/valid

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/valid


In [90]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50) : copyfile(shuf[i], DATA_HOME_DIR + '/sample/valid/' + shuf[i])

In [91]:
# Copied 50 random files from /valid to /sample/valid
# /valid still has 2000 files, /sample/valid has 50
%ls | wc -l
%ls ../sample/valid/ | wc -l

2000
50


## Rearrange files into their respective directories

In [92]:
#Divide cat/dog images into separate directories

%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/sample/train


In [94]:
%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/sample/valid


In [95]:
%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/valid


In [96]:
%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/train


In [97]:
#Create single unknown class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/test


## Finetuning and Training

In [98]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path = DATA_HOME_DIR + '/results/'
train_path = path + '/train/'
valid_path = path + '/valid'

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux


In [99]:
vgg = Vgg16()

In [101]:
#You can experiment with no_of_epochs to improve the model
batch_size = 64
no_of_epochs=3

In [102]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size * 2)
vgg.finetune(batches)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path + latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs

Running epoch 0
Epoch 1/1
  512/23000 [..............................] - ETA: 8717s - loss: 1.0537 - acc: 0.6914